In [ ]:
#!pip install hypernetx

In [ ]:
#!pip install networkx

<img src='images/harrypotter_image.png' width='200px' style="float:left;margin-right:10pt"></img>
# Illustration of Static Hypergraphs using Kaggle's HarryPotter dataset.

In this tutorial we introduce `hypernetx.StaticEntity` and `hypernetx.StaticEntitySet` and the new `static=True` attribute in the `hypernetx.Hypergraph` class. 

Harry Potter Data is available here: https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset.

Python code for parsing the dataset is in `harrypotter.py` in the `hypernetx/utils/toys directory`.

In [ ]:
import hypernetx as hnx
import networkx as nx
import matplotlib.pyplot as plt


## The Harry Potter Dataset: 
To use a csv file for a Static Hypergraph, we need every cell filled with a label. 
We have edited the Harry Potter dataset so that it has 5 categories and every cell is filled. Where a value is unknown, we marked it as "Unknown *category_name*". 

In [ ]:
hogwarts = hnx.HarryPotter()

In [ ]:
hogwarts.dataframe

### We define a labeling based on the categories and store it in an Ordered Dictionary.
The ordering of labels is determined by their order of appearance in the table with the exception of Unknown labels, which are always listed first.

In [ ]:
hogwarts.labels

### Assign unique ids to the label values 
We encode the data in each column of the dataframe using a sequence of integers and store the coded data along with a translator function to retrieve the original names as needed. Here we remove duplicate rows but counts could be collected for a weighting scheme. **Watch for a near future release.**

In [ ]:
## List of nonzero indices
hogwarts.data

In [ ]:
hogwarts.data.shape

## StaticEntity and StaticEntitySet

The entire dataset has now been represented using a data array and a dictionary associating columns and integers with labels and values in the original data.

The basic object in HyperNetX, which holds the data and label dictionary for a static hypergraph, is a `StaticEntity`. Similar to the `hnx.Entity` class, the data structure rests in the background to hold the data for flexibly switching between different orders of containment.

Each column of the data is considered a **level** in the StaticEntity. A level's order corresponds to its column position in the datatable. The column header serves as a key to the label dictionary.

In [ ]:
E = hnx.StaticEntity(data = hogwarts.data, labels = hogwarts.labels)
E.keys

### A StaticEntitySet is a StaticEntity restricted to two levels. 
By default, a StaticEntity will grab the 1st two levels of the data and first two keys of the labels, but any pair of levels may be specified. 

In [ ]:
ES = hnx.StaticEntitySet(E)
ES.labels

## Static Hypergraph
A static hypergraph is one where all nodes and edges are known at the time of construction. This permits an internal ordering and uid structure for easy reference and faster computation of metrics.

**Static Hypegraphs can be instantiated with a StaticEntitySet similar to the way a dynamic hypergrapah can be instantiated using an EntitySet.**




In [ ]:
H = hnx.Hypergraph(ES,static=True,name='Hogwarts')


**But we can also pass the dataframe and specify which columns we want to use as edges and nodes. The default behavior is to use the first two columns.**
$$\text{df}[\text{edge_column},\text{node_column}]$$

In [ ]:
H = hnx.Hypergraph(hogwarts.dataframe)
H.edges,H.nodes

In [ ]:
plt.title('Hogwarts Hypergraph',fontsize=20)
hnx.draw(H)

## General construction of a static Hypergraph:
Set the parameter `static=True` inside the hypergraph constructor and input a set system just as you did before. If the set system is a pandas dataframe, a StaticEntity or a StaticEntitySet, the parameter is automaticaly set to True.

In [ ]:
## example:
simple_data = {'A':{1,2,3},'B':{2,3,4},'C':{3,4,5}}
simple_static_hypergraph = SSH = hnx.Hypergraph(simple_data, static=True)
hnx.draw(SSH)

In [ ]:
SSH.isstatic

#### Static Hypergraphs are immutable. You can't add or remove nodes or edges. Uncomment the last line below and try it:


In [ ]:
## Static Hypergraphs are immutable. You can't add or remove nodes or edges.
## Uncomment the last line and try it:
new_edge = hnx.Entity('D',[4,5,6])
# SSH.add_edge(new_edge)

In [ ]:
## But you can remove the static property and create a new hypergraph 
## This will also remove the benefits of an immutable datastructure and may slow things down.
SSH = SSH.remove_static()
SSH.add_edge(new_edge)

## State Dictionary
Since a static hypergraph does not lose nodes and edges, metrics computed on the hypergraph will persist. We store them in a state dictionary. 

Let $H$ be the Hogwarts hypergraph we constructed above.

In [ ]:
H.state_dict

### The output of certain methods are automatically stored in the state dictionary once they are computed.

In [ ]:
H.incidence_matrix().todense()
H.state_dict

### They can be retrieved by their keys. But will automatically be retrieved when the method is called again to avoid duplicating the computation.

In [ ]:
H.state_dict['incidence_matrix'].todense()

## Most Hypergraph methods apply to Static Hypergraphs
Any method, which does not change the data and labels of the underlying StaticEntitySet, can be used by the static Hypergraph.

In [ ]:
H.dataframe()

### Restrict to specific edges and nodes

In [ ]:
HF = H.restrict_to_edges(['Gryffindor','Ravenclaw','Slytherin','Hufflepuff'])
HF.dataframe()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,6))
hnx.draw(H,ax=ax[0]);
hnx.draw(H.dual())
H.edges

## Collapse identical elements
This method exists to collapse identical nodes and edges and is implemented for dynamic hypergraphs.
We wish to do the same for large unwieldy hypergraphs stored as static.

In [ ]:
pos = {'Unknown House': [-0.11,  0.4 ],
 'Gryffindor': [-0.32,  0.27],
 'Ravenclaw': [0.57, 0.27],
 'Hufflepuff': [-0.02,  0.16],
 'Slytherin': [-0.02, -0.51],
 'Durmstrang Institute': [-0.09, -1.  ],
 'Unknown Blood status': [0.15, 0.66],
 'Half-blood': [0.24, 0.04],
 'Pure-blood': [-0.45, -0.08],
 'Pure-blood or half-blood': [ 0.05, -0.21]}

In [ ]:
nodes = ['Pure-blood or half-blood',  'Unknown Blood status', 'Pure-blood', 'Half-blood',  ]
Hn = H.restrict_to_nodes(nodes)
hnx.draw(Hn,pos=pos)

In [ ]:
Hc,clses = Hn.collapse_edges(return_equivalence_classes=True)

fig,ax = plt.subplots(1,2,figsize=(15,6))
hnx.draw(Hn,ax=ax[0],pos=pos);
ax[0].set_title('original',fontsize=20,color='r')
hnx.draw(Hc,ax=ax[1],pos=pos);
ax[1].set_title('collapsed',fontsize=20,color='r');
clses

### More hypergraph methods

In [ ]:
## bipartite
G = H.bipartite() ## this is a NetworkX graph
cmap = ['r' if G.nodes[n]['bipartite']==0 else 'cyan' for n in G.nodes ]
top = nx.bipartite.sets(G)[0]
pos = nx.bipartite_layout(G, top)
nx.draw(H.bipartite(),node_color=cmap,with_labels=True, pos=pos)

In [ ]:
## reporting
print(hnx.info(H))

In [ ]:
## Once the dist stats are computed, they are stored in the state dict for fast recall and reference
hnx.dist_stats(H)

In [ ]:
H.state_dict

In [ ]:
## toplexes
fig,ax = plt.subplots(1,2,figsize=(15,6))
pos = hnx.draw(H,ax=ax[0],return_pos=True)
hnx.draw(H.toplexes(),ax=ax[1],pos=pos)